In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
import seaborn as sns
from sklearn.datasets import load_boston
import pandas as pd 
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict,train_test_split, KFold, ShuffleSplit


%matplotlib notebook 



## Cross Validation

### Train- Test Split

In order to tune the model to find the structure in the data in the best possible fashion, we must use the existing data to identify settings for the model’s parameters that yield the best and most realistic predictive performance.

Traditionally, this has been achieved by splitting the existing data into training and test sets. The training set is used
to build and tune the model and the test set is used to estimate the model’s predictive performance. Modern approaches to model building split the data into multiple training and testing sets, which have been shown to often find more optimal tuning parameters and give a more accurate representation of the model’s predictive performance.

To avoid over-fitting, we use a general model building approach that encompasses model tuning and model evaluation
with the ultimate goal of finding the reproducible structure in the data. This approach entails splitting existing data into distinct sets for the purposes of tuning model parameters and evaluating model performance. The choice of data splitting method depends on characteristics of the existing data such as its size and structure.

Below is an example of how test train split can be done in python
 
take the code below and run it in the cell- 

```python 


#load boston dataset using boston_dataset.feature_names
boston_dataset = datasets.load_boston()
boston_data = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston_data['MEDV'] = boston_dataset.target

#create X and y as input and output vectors
X = boston_data[['CRIM', 'ZN', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'PTRATIO', 'B', 'LSTAT']]
y = boston_data[['MEDV']]

# Write code below to create train-test split
#Use sklearn.modelselection. train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)
X_train.head()

```



In [2]:
# run test train split code here 


In the above, example we took the boston dataset and broke it down into a test set with 30% of the examples and 70% of the examples in the training set. This breakdown allows us to train the model on the training set and run it on the test set. One should never train the model on examples in the test set. 

What does that mean? 

It means that for example, in the context of what we have learned so far, we should never run the fit function on the test set data in linear regression. Just the predict function. 

We expect the test set metric scores, accuracy or mean squared error(in case of regression) to lower than than the training set accuracy score. This is because the test set may have data the model has not seen before.  For example, if your house price variation is say 10,000 dollars based on location and the test set has examples where that variation is far larger then the predicted value of the house price will be lower than the actual value in the test set. Hence, the mean squared error will be higher in the test set. 

## K - fold cross validation

Now what if the test set of example that the train data does not contain at all? For example suppose you are training a model to fit the boston housing dataset and the test set has house prices out of the range of the the training set entirely. Also, suppose these are not outliers. Since we are doing a random train-test split this can easily happen. Also for smaller datasets, this can definitely happen where your model may miss some key examples that it can learn from due to the test train split. 

In such scenarios, we can use K-fold cross validation. K-fold cross validation is useful especially when 

- You have a small dataset and partation into test and train is not feasible<br>
- you want to check how much variation is there in the model <br>

So what is k fold cross validation- Rather than splitting the data into a single test train split, we split it into multiple sets. For example, we may choose to split the dataset into 5 sets. We set the first set as the validation set and train a model. Then we choose the second set as the validation set and train a model on that and so on. Finally, iterating through the sets, 5 models will have been trained and subsequently the accuracies and the error from the 5 validation sets will also be availaible. On this basis we can easily check if the variation major or minor. If for example, for the 5 models we have an $r^2$ score of (0.8,0.9,0.87,0.89,0.91) then the model variation is minor. Suppose in one of the models the $r^2$ score is really tiny like 0.1 then we know that there is something wrong with that specific set of training data. In such cases we can go back and check our training data to make sure that there are no errors. 

The benefit of k fold cross validation is that it allows us to build a model using all the data that we have and especially with models which can be trained quickly cross validation works well. 



to show this we will show the example of running K- fold cross validation on an examples of linear regression with the Shampoo dataset and the Boston housing dataset. 

# K fold cross - Shampoo sales dataset


Run the code below to perform k fold cross validation on the shampoo sales dataset. (This original dataset is credited to Makridakis, Wheelwright, and Hyndman (1998))

```Python
# import the shampoo sales dataset 
shampoo_data = pd.read_csv("shampoo_sales.csv")
sales_full = np.array(shampoo_data["Sales"].tolist()) 
sales_full_reshaped = sales_full.reshape(sales_full.shape[0], 1)
x_values_full = np.arange(0, sales_full.shape[0])
x_values_full_reshaped = x_values_full.reshape(x_values_full.shape[0],1)

lr_cv = LinearRegression()
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=1)
cross_val_score(lr_cv, x_values_full_reshaped, sales_full_reshaped , cv=cv, scoring="r2")
```







In [3]:
# copy and run the code for k fold cross validation on shampoo dataset here 


Notice you did the model building a bit more differently here. Rather than the straight out linear regression we import cross_val_score from sklearn.model_selection and then run our model. Another important line to pay attention to is the definition of cv. Unlike the sklearn documentation we did not definite cv as a number. We used 'ShuffleSplit' to provide values for cv. This is done because without shuffling the data cross validation will not yield appropriate values. 

Question: What happens when you remove the shuffle split line from the code and run the cross validation code for 'cv=5' ? 


In [4]:
# run code here


In [5]:
# solution
# import the shampoo sales dataset 
shampoo_data = pd.read_csv("shampoo_sales.csv")
sales_full = np.array(shampoo_data["Sales"].tolist()) 
sales_full_reshaped = sales_full.reshape(sales_full.shape[0], 1)
x_values_full = np.arange(0, sales_full.shape[0])
x_values_full_reshaped = x_values_full.reshape(x_values_full.shape[0],1)

lr_cv = LinearRegression()
cross_val_score(lr_cv, x_values_full_reshaped, sales_full_reshaped , cv=5, scoring="r2")

# you should see negative values in the k fold accuracies since k fold is not shuffing the data 


array([-8.79471507, -0.74214013, -0.01991722, -0.4458732 , -1.77163797])

We can actually predict the values for a given model using the 'cross_val_predict' function. Rather than running 'cross_val_score' we run the predict function- 


```Python 
shampoo_data = pd.read_csv("shampoo_sales.csv")
sales_full = np.array(shampoo_data["Sales"].tolist()) 
sales_full_reshaped = sales_full.reshape(sales_full.shape[0], 1)
x_values_full = np.arange(0, sales_full.shape[0])
x_values_full_reshaped = x_values_full.reshape(x_values_full.shape[0],1)

lr_cv = LinearRegression()
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=1)
cross_val_predict(lr_cv, x_values_full_reshaped, sales_full_reshaped , cv=cv, scoring="r2")

```


In [ ]:
# run cross val predict code here 
shampoo_data = pd.read_csv("shampoo_sales.csv")

sales_full = np.array(shampoo_data["Sales"].tolist()) 
sales_full = np.ndarray.flatten(sales_full)
sales_full_reshaped = sales_full.reshape(sales_full.shape[0], 1)

x_values_full = np.arange(0, sales_full.shape[0])
x_values_full = np.ndarray.flatten(x_values_full)
x_values_full_reshaped = x_values_full.reshape(x_values_full.shape[0],1)

lr_pr = LinearRegression()
cross_val_predict(lr_pr, x_values_full_reshaped, sales_full_reshaped, cv=5)


Note: Cross validate predict does not work with 'ShuffleSplit'  so make sure that you shuffle your dataset before you cross validate predict it. 



We can also do cross validation manually. Sklearn allows you to th break up the data using a function called 'KFold' 

run the code below to break up the data and get individual sets. 

```Python

# run the code above for generating row incidices from k fold cross validation 

kf = KFold(n_splits=3, random_state=1, shuffle=True)
kf.get_n_splits(shampoo_data.values)

for counter, indices in enumerate(kf.split(shampoo_data.values)):
    train_index ,validation_index= indices
    print("cross validation iteration {} \nvalidation indices: \n {} \ntrain indices: \n {}".format(counter, validation_index, train_index))
    
```
**Output**
```python 
cross validation iteration 0 
validation indices: 
 [ 3 17 19 21 23 26 27 28 29 30 33 34] 
train indices: 
 [ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 18 20 22 24 25 31 32 35]
cross validation iteration 1 
validation indices: 
 [ 2  4  6 10 14 18 20 22 24 25 31 32] 
train indices: 
 [ 0  1  3  5  7  8  9 11 12 13 15 16 17 19 21 23 26 27 28 29 30 33 34 35]
cross validation iteration 2 
validation indices: 
 [ 0  1  5  7  8  9 11 12 13 15 16 35] 
train indices: 
 [ 2  3  4  6 10 14 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34]

````

when you print the test and the train index fo

In [7]:
# run the code above for generating row incidices from k fold cross validation 


What we did above is generate the row indices using which we can preform cross validation. That is what is next expected of you- 

Question: using the code above, generate the validation sets and training sets for the shampoo datasets



In [8]:
# solution 
kf = KFold(n_splits=3, random_state=1, shuffle=True)
kf.get_n_splits(shampoo_data.values)

for counter, indices in enumerate(kf.split(shampoo_data.values)):
    train_index ,validation_index= indices
    
    train_rows = shampoo_data.iloc[train_index]
    validation_rows = shampoo_data.iloc[validation_index]
    
    print("train rows: \n {} \n validation rows \n {}".format(train_rows, validation_rows))

    

train rows: 
    Month  Sales
0   1-01  266.0
1   1-02  145.9
2   1-03  183.1
4   1-05  180.3
5   1-06  168.5
6   1-07  231.8
7   1-08  224.5
8   1-09  192.8
9   1-10  122.9
10  1-11  336.5
11  1-12  185.9
12  2-01  194.3
13  2-02  149.5
14  2-03  210.1
15  2-04  273.3
16  2-05  191.4
18  2-07  226.0
20  2-09  289.9
22  2-11  264.5
24  3-01  339.7
25  3-02  440.4
31  3-08  407.6
32  3-09  682.0
35  3-12  646.9 
 validation rows 
    Month  Sales
3   1-04  119.3
17  2-06  287.0
19  2-08  303.6
21  2-10  421.6
23  2-12  342.3
26  3-03  315.9
27  3-04  439.3
28  3-05  401.3
29  3-06  437.4
30  3-07  575.5
33  3-10  475.3
34  3-11  581.3
train rows: 
    Month  Sales
0   1-01  266.0
1   1-02  145.9
3   1-04  119.3
5   1-06  168.5
7   1-08  224.5
8   1-09  192.8
9   1-10  122.9
11  1-12  185.9
12  2-01  194.3
13  2-02  149.5
15  2-04  273.3
16  2-05  191.4
17  2-06  287.0
19  2-08  303.6
21  2-10  421.6
23  2-12  342.3
26  3-03  315.9
27  3-04  439.3
28  3-05  401.3
29  3-06  437.4
30  3-07

In this notebook we saw how to do cross validation with sklearn in two ways. There are other functions that will be useful to learn  when using cross validation- 

some function worth exploring are - 

1) LogisticRegressionCV where you can logistic regression with cross validation. 
2) There are different types of cross validation that you can do for example-
    - GroupKFold
    - StratifiedKFold 
  
Now that you have a basic idea of how to implement regular cross validation, it will be easy to look at the sklearn documentation so that you can better understand how well to implement other cross validation methods. 
 